In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)

In [8]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [9]:
dataloader = DataLoader(dataset, batch_size=20, num_workers=2)

In [10]:
videos, questions, answers, hidden_states, vid_folder  = iter(dataloader).next()

In [13]:
dataset.dataframe.head(10)

,Unnamed: 0,imgs_folder_path,box_x,pipe_x,enemy_speed,mario_speed,answer_box,answer_pipe,answer_enemy
0,0,data/imgs_series/00001,0.156313,0.185930,1.000000,77.490709,2.426097,9.639865,3.860908
1,1,data/imgs_series/00002,0.214429,0.145729,0.250000,80.115156,2.708601,9.224222,2.239635
2,2,data/imgs_series/00003,0.018036,0.130653,0.666667,95.058088,1.251866,7.742634,2.034848
3,3,data/imgs_series/00004,0.104208,1.000000,0.291667,62.359386,2.597845,14.576795,3.623262
4,4,data/imgs_series/00005,0.262525,0.396985,0.125000,51.039479,4.721835,15.458622,4.774413
5,5,data/imgs_series/00006,0.797595,0.477387,0.041667,65.566828,7.747820,12.277550,2.780107
6,6,data/imgs_series/00007,0.038076,0.376884,0.083333,63.341972,2.036564,12.393046,3.026803
7,7,data/imgs_series/00008,0.689379,0.095477,0.750000,76.515466,5.933441,9.527486,3.282067
8,8,data/imgs_series/00009,0.390782,0.206030,0.708333,59.868291,5.094517,12.544203,6.156157
9,9,data/imgs_series/00010,0.150301,0.366834,0.416667,95.022940,1.946898,8.240116,1.832633


In [15]:
const.

<module 'src.constants' from '/Users/henrik/Documents/hsu/mario-communicating-agents/src/constants.py'>

In [9]:
checkpoint_path = './lightning_logs/version_6/checkpoints/epoch=25-step=207.ckpt'

In [10]:
model = LitModule.load_from_checkpoint(checkpoint_path)

In [11]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [12]:
dataloader = DataLoader(dataset, batch_size=10, num_workers=2)

In [13]:
videos, questions, answers, hidden_states, _  = iter(dataloader).next()

In [14]:
predictions = model.eval()(videos)
predictions

tensor([[0.2236, 0.4662, 0.9885],
        [0.2244, 0.4721, 0.2638],
        [0.0537, 0.4881, 0.6125],
        [0.1221, 0.4824, 0.3305],
        [0.2819, 0.4582, 0.0720],
        [0.9013, 0.5235, 0.1032],
        [0.1281, 0.4774, 0.0551],
        [0.8868, 0.5278, 0.6848],
        [0.4278, 0.5030, 0.6455],
        [0.1881, 0.4996, 0.4343]], grad_fn=<AddmmBackward>)

In [15]:
hidden_states

tensor([[0.1563, 0.1859, 1.0000],
        [0.2144, 0.1457, 0.2500],
        [0.0180, 0.1307, 0.6667],
        [0.1042, 1.0000, 0.2917],
        [0.2625, 0.3970, 0.1250],
        [0.7976, 0.4774, 0.0417],
        [0.0381, 0.3769, 0.0833],
        [0.6894, 0.0955, 0.7500],
        [0.3908, 0.2060, 0.7083],
        [0.1503, 0.3668, 0.4167]], dtype=torch.float64)

In [18]:
mse_loss = torch.nn.MSELoss(reduction='sum')
mse_hidden = mse_loss(predictions[0:2,:].type(torch.float32),
                      hidden_states[0:2,:].type(torch.float32))
mse_hidden

tensor(0.2628, grad_fn=<MseLossBackward>)

73323.68388475002

In [ ]:
model.eval()(videos)

In [22]:
predictions[0:2,:]

tensor([[543.4559, 996.8821,  37.2706],
        [543.4633, 996.8958,  37.2711]], grad_fn=<SliceBackward>)